Transformaciones elemento-a-elemento
======================================

#### Generan un nuevo RDD a partir de uno dado

-   `filter(func)` filtra los elementos de un RDD

In [2]:
# Obtener los valores positivos de un rango de números
from __future__ import print_function
from test_helper import Test
rdd = sc.parallelize(xrange(-5,5))          # Rango [-5, 5)
filtered_rdd = rdd.filter(lambda x: x >= 0)   # Devuelve los positivos

Test.assertEquals(filtered_rdd.collect(), [0, 1, 2, 3, 4])

In [3]:
// Obtener los valores positivos de un rango de números
val rdd = sc.parallelize(-5 to 5)         // Rango [-5, 5]
val filtered_rdd = rdd.filter(x => x>=0)  // Devuelve los positivos

assert(filtered_rdd.collect().sameElements(Array(0, 1, 2, 3, 4, 5)))

 
-   `map(func)` aplica una función a los elementos de un RDD

In [5]:
# Ejemplo en PySpark
# Añade 1 a cada elemento del RDD
# Para cada elemento, obtiene una tupla (x, x**2)
def add1(x):
    return(x+1)

squared_rdd = (filtered_rdd
               .map(add1)                 # Añade 1 a cada elemento del RDD
               .map(lambda x: (x, x*x)))  # Para cada elemento, obtén una tupla (x, x**2)

Test.assertEquals(squared_rdd.collect(), [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)])

In [6]:
/* Añade 1 a cada elemento del RDD
   Para cada elemento, obtén una tupla (x, x**2) */
def add1(x: Int): Int = return(x+1)

val squared_rdd = (filtered_rdd
               .map(add1)
               .map(x => (x, x*x))) 
               
assert(squared_rdd.collect().sameElements(Array((1,1), (2,4), (3,9), (4,16), (5,25), (6,36))))

-   `flatMap(func)` igual que `map`, pero “aplana” la salida

In [8]:
squaredflat_rdd = (filtered_rdd
                   .map(add1)
                   .flatMap(lambda x: (x, x*x)))  # Da la salida en forma de lista
                   
Test.assertEquals(squaredflat_rdd.collect(), [1, 1, 2, 4, 3, 9, 4, 16, 5, 25])

In [9]:
val squaredflat_rdd = (filtered_rdd
                       .map(add1)
                       .flatMap(x => List(x, x*x)))  // Da la salida en forma de lista
                       
assert(squaredflat_rdd.collect().sameElements(Array(1, 1, 2, 4, 3, 9, 4, 16, 5, 25, 6, 36)))

-   `sample(withReplacement, fraction, seed=None)` devuelve una muestra del RDD
    - `withReplacement` - si True, cada elemento puede aparecer varias veces en la muestra
    - `fraction` - tamaño esperado de la muestra como una fracción del tamaño del RDD 
        -  **sin reemplazo**: probabilidad de seleccionar un elemento, su valor debe ser [0, 1]
        -  **con reemplazo**: número esperado de veces que se escoge un elemento, su valor debe ser >= 0
    - `seed` - semilla para el generador de números aleatorios


In [11]:
srdd1 = squaredflat_rdd.sample(False, 0.5)
srdd2 = squaredflat_rdd.sample(True, 2)
srdd3 = squaredflat_rdd.sample(False, 0.8, 14)
print('s1={0}\ns2={1}\ns3={2}'.format(srdd1.collect(), srdd2.collect(), srdd3.collect()))

In [12]:
val s1 = squaredflat_rdd.sample(false, 0.5).collect()
val s2 = squaredflat_rdd.sample(true, 2).collect()
val s3 = squaredflat_rdd.sample(false, 0.8).collect()

-   `distinct()` devuelve un nuevo RDD sin duplicados
    - El orden de la salida no está definido

In [14]:
distinct_rdd = squaredflat_rdd.distinct()
print(distinct_rdd.collect())

In [15]:
val distinct_rdd = squaredflat_rdd.distinct()
println(distinct_rdd.collect().mkString(" "))

-   `groupBy(func)` devuelve un RDD con los datos agrupados en formato clave/valor, 
usando una función para obtener la clave

In [17]:
grouped_rdd = distinct_rdd.groupBy(lambda x: x%3)
print(grouped_rdd.collect())
print([(x,sorted(y)) for (x,y) in grouped_rdd.collect()])

In [18]:
val grouped_rdd = distinct_rdd.groupBy(x => x%3)
val grouped = grouped_rdd.collect()
grouped.foreach(f => println(f._1+" ("+f._2.mkString(" ")+") "))

### Transformaciones sobre dos RDDs

Operaciones tipo conjunto sobre dos RDDs


-   `rdda.union(rddb)` devuelve un RDD con los datos de los dos de partida

In [21]:
rdda = sc.parallelize(['a', 'b', 'c'])
rddb = sc.parallelize(['c', 'd', 'e'])
rddu = rdda.union(rddb)
Test.assertEquals(rddu.collect(),['a', 'b', 'c', 'c', 'd', 'e'])

In [22]:
val rdda = sc.parallelize(List('a', 'b', 'c'))
val rddb = sc.parallelize(List('c', 'd', 'e'))
val rddu = rdda.union(rddb)
assert(rddu.collect().sameElements(Array('a', 'b', 'c', 'c', 'd', 'e')))

- `rdda.intersection(rddb)` devuelve un RDD con los datos comunes en ambos RDDs

In [24]:
rddi = rdda.intersection(rddb)
Test.assertEquals(rddi.collect(),['c'])

In [25]:
val rddi = rdda.intersection(rddb)
assert(rddi.collect().sameElements(Array('c')))

-   `rdda.subtract(rddb)` devuelve un RDD con los datos del primer RDD menos los del segundo

In [27]:
rdds = rdda.subtract(rddb)
Test.assertEquals(rdds.collect(), ['a', 'b'])

In [28]:
val rdds = rdda.subtract(rddb)
assert(rdds.collect().sameElements(Array('a', 'b')))

`rdda.cartesian(rddb)` producto cartesiano de ambos RDDs (operación muy costosa)

In [30]:
rddc = rdda.cartesian(rddb)
Test.assertEquals(rddc.collect(), 
                  [('a','c'),('a','d'),('a','e'),('b','c'),('b','d'),('b','e'),('c','c'), ('c','d'), ('c','e')])

In [31]:
val rddc = rdda.cartesian(rddb)
assert(rddc.collect().sameElements( 
                  Array(('a','c'),('a','d'),('a','e'),('b','c'),('b','d'),('b','e'),('c','c'), ('c','d'), ('c','e'))))

Acciones sobre RDDs simples
======================================

#### Obtienen datos (simples o compuestos) a partir de un RDD


#### Principales acciones de agregación: `reduce` y `fold`

-   `reduce(op)` combina los elementos de un RDD en paralelo, aplicando un operador
    - El operador de reducción debe ser un *monoide conmutativo* (operador binario asociativo y conmutativo)
    - Primero se realiza la redución a nivel de partición y luego se van reduciendo los valores intermedios

In [35]:
rdd = sc.parallelize(xrange(1,10), 8)  # rango [1, 10)
print(rdd.glom().collect())

# Reducción con una función lambda
p = rdd.reduce(lambda x,y: x*y) # r = 1*2*3*4*5*6*7*8*9 = 362880
print("1*2*3*4*5*6*7*8*9 = {0}".format(p))

# Reducción con un operador predefinido
from operator import add
s = rdd.reduce(add) # s = 1+2+3+4+5+6+7+8+9 = 45
print("1+2+3+4+5+6+7+8+9 = {0}".format(s))

# Prueba con un operador no conmutativo
p = rdd.reduce(lambda x,y: x-y) # r = 1-2-3-4-5-6-7-8-9 = -43
print("1-2-3-4-5-6-7-8-9 = {0}".format(p))

# No funciona con RDDs vacíos
#sc.parallelize([]).reduce(add)


In [36]:
val rdd = sc.parallelize(1 to 9)    // rango [1, 9]
val l = rdd.glom().collect()

// Reducciones conn funciones anónimas
val p = rdd.reduce((x,y) => x*y)    // r = 1*2*3*4*5*6*7*8*9 = 362880
val s = rdd.reduce(_+_)             // s = 1+2+3+4+5+6+7+8+9 = 45

// No funciona con RDDs vacíos
//sc.parallelize(List[Int]()).reduce(_+_)

-   `fold(cero, op)` versión general de `reduce`: 
    - Debemos proporcionar un valor inicial `cero` para el operador
    - El valor inicial debe ser el valor identidad para el operador (p.e. 0 para suma; 1 para producto, o una lista vacía para concatenación de listas)
        - Permite utilizar RDDs vacíos
    - La función `op` debe ser un monoide conmutativo para garantizar un resultado consistente
        - Comportamiento diferente a las operaciones `fold` de lenguajes como Scala
        - El operador se aplica a nivel de partición (usando `cero` como valor inicial), y finalmente entre todas las particiones (usando `cero`de nuevo)
        - Para operadores no conmutativos el resultado podría ser diferente del obtenido mediante un `fold` secuencial

In [38]:
rdd = sc.parallelize([range(1,5), range(-10,-3), ['a', 'b', 'c']])
print(rdd.glom().collect())

f = rdd.fold([], lambda x,y: x+y)
print(f)

# Se puede hacer un fold de un RDD vacío
sc.parallelize([]).fold(0, add)


In [39]:
val rdd = sc.parallelize(List(List.range(1,5), List.range(-10,-3), List('a', 'b', 'c')))
val l = rdd.glom().collect()

val f = rdd.fold(List[AnyVal]())((l1, l2) => l1 ::: l2)
println(f)

// Se puede hacer un fold de un RDD vacío
sc.parallelize(List()).fold(0)(_+_)


In [40]:
// Ejemplo de comportamiento extraño
// Comportamiento en secuencial
val a = Array(2, 3)
val f = a.fold(0)((p, v) => p+v*v) // f = (0 + 2*2) + (3*3) = 13

// Comportamiento en paralelo
val rdda = sc.parallelize(a, 2)
val fa = rdda.fold(0)((p, v) => p+v*v)  
// Partición 0: fa0 = (0 + 2*2) = 4
// Partición 1: fa1 = (0 + 3*3) = 9
// fold final: fa = (0 + fa0*fa0) + (fa1*fa1) = (0 + 4*4) + (9*9) = 97

// El resultado puede variar con el número de particiones
val rdda = sc.parallelize(a, 1)
val fa = rdda.fold(0)((p, v) => p+v*v)
// ¿Por qué obtenemos este valor?

#### Otras acciones de agregación: `aggregate`


  - `aggregate(cero,seqOp,combOp)`: Devuelve una colección agregando los elementos del RDD usando dos funciones:
    1. `seqOp` -  agregación a nivel de partición: se crea un acumulador por partición (inicializado a `cero`) y se agregan los valores de la partición en el acumulador
    2. `combOp` - agregación entre particiones: se agregan los acumuladores de todas las particiones
    -  Ambas agregaciones usan un valor inicial `cero` (similar al caso de `fold`).
 - Versión general de `reduce` y `fold`    
 - La primera función (`seqOp`) puede devolver un tipo, U, diferente del tipo T de los elementos del RDD
    - `seqOp` agregar datos de tipo T y devuelve un tipo U
    - `combOp` agrega datos de tipo U
    - `cero` debe ser de tipo U
 - Permite devolver un tipo diferente al de los elementos del RDD de entrada.

In [43]:
l = [1, 2, 3, 4, 5, 6, 7, 8]
rdd = sc.parallelize(l)

# acc es una tupla de tres elementos (List, Double, Int)
# En el primer elemento de acc (lista) le concatenamos los elementos del RDD al cuadrado
# en el segundo, acumulamos los elementos del RDD usando multiplicación
# y en el tercero, contamos los elementos del RDD
seqOp  = (lambda acc, val: (acc[0]+[val*val], 
                            acc[1]*val, 
                            acc[2]+1))
# Para cada partición se genera una tupla tipo acc
# En esta operación se combinan los tres elementos de las tuplas
combOp = (lambda acc1, acc2: (acc1[0]+acc2[0], 
                              acc1[1]*acc2[1], 
                              acc1[2]+acc2[2]))
                              
a = rdd.aggregate(([], 1., 0), seqOp, combOp) 

print(a)

Test.assertEquals(a[1], 8.*7.*6.*5.*4.*3.*2.*1.)
Test.assertEquals(a[2], len(l))


In [44]:
val l = List(1, 2, 3, 4, 5, 6, 7, 8)
val rdd = sc.parallelize(l)
val dl = rdd.glom.collect()

// acc es una tupla de tres elementos (List[Int], Double, Int)
// En el primer elemento de acc, insertamos por el principio los elementos del RDD al cuadrado
// en el segundo, acumulamos los elementos del RDD usando multiplicación
// y en el tercero, contamos los elementos del RDD
def seqOp(acum: (List[Int], Double, Int), valor: Int)  = ((valor*valor) :: acum._1, 
                                                          acum._2 * valor, 
                                                          acum._3 + 1)
// Para cada partición se genera una tupla tipo acc
// En esta operación se combinan los tres elementos de las tuplas
def combOp(acum1: (List[Int], Double, Int), acum2: (List[Int], Double, Int)) = (acum1._1 ::: acum2._1, 
                                                                                acum1._2 * acum2._2, 
                                                                                acum1._3 + acum2._3)

val a = rdd.aggregate((List[Int](), 1.0, 0))(seqOp, combOp)

assert(a._2 == (8.0*7.0*6.0*5.0*4.0*3.0*2.0*1.0))
assert(a._3 == l.length)

#### Acciones para contar elementos
- `count()` devuelve un entero con el número exacto de elementos del RDD
- `countApprox(timeout, confidence=0.95)` versión aproximada de `count()` que devuelve un resultado potencialmente incompleto en un tiempo máximo, incluso si no todas las tareas han finalizado. (Experimental).
    - `timeout` es un entero largo e indica el tiempo en milisegundos
    - `confidence` probabilidad de obtener el valor real. Si `confidence` es 0.90 quiere decir que si se ejecuta múltiples veces, se espera que el 90% de ellas se obtenga el valor correcto. Valor [0,1]
- `countApproxDistinct(relativeSD=0.05)` devuelve una estimación del número de elementos diferentes del RDD.  (Experimental).
    - `relativeSD` – exactitud relativa (valores más pequeños implican menor error, pero requieren más memoria; debe ser mayor que 0.000017).


In [46]:
rdd = sc.parallelize([i % 20 for i in xrange(10000)], 16)
print("Número total de elementos: {0}".format(rdd.count()))
print("Número de elementos distintos: {0}".format(rdd.distinct().count()))

print("Número total de elementos (aprox.): {0}".format(rdd.countApprox(1, 0.4)))
print("Número de elementos distintos (approx.): {0}".format(rdd.countApproxDistinct(0.5)))

-   `countByValue()` devuelve el número de apariciones de cada elemento del RDD como un mapa (o diccionario) de tipo clave/valor
    - Las claves son los elementos del RDD y cada valor, el número de ocurrencias de la clave asociada al mismo      

In [48]:
rdd = sc.parallelize(list("abracadabra")).cache()
mimapa = rdd.countByValue()

print(type(mimapa))
print(mimapa.items())

#### Acciones para obtener valores
- Estos métodos deben usarse con cuidado, si el resultado esperado es muy grande puede saturar la memoria del driver

-   `collect()` devuelve una lista con todos los elementos del RDD

In [51]:
lista = rdd.collect()
print(lista)

-   `take(n)` devuelve los `n` primeros elementos del RDD
-   `takeSample(withRep, n, [seed])` devuelve `n` elementos aleatorios del RDD
    - `withRep`: si True, en la muestra puede aparecer el mismo elemento varias veces
    - `seed`: semilla para el generador de números aleatorios

In [53]:
t = rdd.take(4)
print(t)
s = rdd.takeSample(False, 4)
print(s)


-   `top(n)` devuelve una lista con los primeros `n` elementos del RDD ordenados en orden descendente
-   `takeOrdered(n,[orden])` devuelve una lista con los primeros `n` elementos del RDD en orden ascendente (opuesto a `top`), o siguiendo el orden indicado en la función opcional

In [55]:
rdd = sc.parallelize([8, 4, 2, 9, 3, 1, 10, 5, 6, 7]).cache()

print("4 elementos más grandes: {0}".format(rdd.top(4)))

print("4 elementos más pequeños: {0}".format(rdd.takeOrdered(4)))

print("4 elementos más grandes: {0}".format(rdd.takeOrdered(4, lambda x: -x)))